In [1]:
import pandas
import datetime
import os
import sys
module_path = os.path.abspath(os.path.join('../data_pipeline/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import sqlite3

/usr/local/lib/python2.7/dist-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/usr/local/lib/python2.7/dist-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/usr/local/lib/python2.7/dist-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/usr/local/lib/python2.7/dist-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/usr/local/lib/python2.7/dist-packages/pandas/core/indexes/base.py:7: RuntimeWarning: numpy.dtype size changed, may indicate bi

In [5]:
ae_dbdir = "/home/bharat/Documents/data/ss_onset_dataset/data/sqlite3/"
ae_db_name = "au_al_ae.sqlite"
omn_table_name = "aualae"
start_date = datetime.datetime(1996,1,1)
end_date = datetime.datetime(1996,12,31)
# set some cutoffs for AL/AE indices
alSSCutoff = -100

In [3]:
conn = sqlite3.connect(ae_dbdir + ae_db_name,
                       detect_types = sqlite3.PARSE_DECLTYPES)
# load data to a dataframe
command = "SELECT * FROM {tb} " +\
                  "WHERE datetime BETWEEN '{stm}' and '{etm}'"
command = command.format(tb=omn_table_name,\
                         stm=start_date, etm=end_date)
aulDF = pandas.read_sql(command, conn)

In [4]:
aulDF[ aulDF["al"] > alSSCutoff ]

,datetime,au,al,ae
525595,1996-12-30 23:55:00,58.0,-7.0,65.0
525596,1996-12-30 23:56:00,60.0,-14.0,74.0
525597,1996-12-30 23:57:00,54.0,-17.0,71.0
525598,1996-12-30 23:58:00,49.0,-16.0,65.0
525599,1996-12-30 23:59:00,46.0,-12.0,58.0
